In [1]:
# %%
from ProtDiffusion.training_utils import VAETrainingConfig, make_clustered_dataloader, set_seed, VAETrainer, count_parameters
from transformers import PreTrainedTokenizerFast

from datasets import load_from_disk

from ProtDiffusion.models.autoencoder_kl_1d import AutoencoderKL1D

import os

seed = 42
set_seed(seed) # Set the random seed for reproducibility

# dataset = load_from_disk('/work3/s204514/UniRef50_grouped')
dataset = load_from_disk('/home/kkj/ProtDiffusion/datasets/UniRef50_grouped')
dataset = dataset.shuffle(seed)

# %%
# tokenizer = PreTrainedTokenizerFast.from_pretrained("/zhome/fb/0/155603/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.1")
# tokenizer = PreTrainedTokenizerFast.from_pretrained("/home/kkj/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.1")
tokenizer = PreTrainedTokenizerFast.from_pretrained("/home/kkj/ProtDiffusion/ProtDiffusion/tokenizer/tokenizer_v4.2")

# Split the dataset into train and temp sets using the datasets library
train_test_split_ratio = 0.0002
train_val_test_split = dataset.train_test_split(test_size=train_test_split_ratio, seed=seed)
train_dataset = train_val_test_split['train']
temp_dataset = train_val_test_split['test']

# Split the temp set into validation and test sets using the datasets library
val_test_split_ratio = 0.5
val_test_split = temp_dataset.train_test_split(test_size=val_test_split_ratio, seed=seed)
val_dataset = val_test_split['train']
test_dataset = val_test_split['test']

# Check dataset lengths
print(f"Train dataset length: {len(train_dataset)}")
print(f"Validation dataset length: {len(val_dataset)}")
print(f"Test dataset length: {len(test_dataset)}")

Random seed set as 42


Loading dataset from disk:   0%|          | 0/149 [00:00<?, ?it/s]

Train dataset length: 42803651
Validation dataset length: 4281
Test dataset length: 4282


In [2]:
mega_batch = 4
batch_size = 2
max_len_start = 64
max_len = max_len_start

print("num cpu cores:", os.cpu_count())
print("setting num_workers to 16")
num_workers = 2

train_dataloader = make_clustered_dataloader(batch_size,
                                             mega_batch,
                                             train_dataset,
                                             tokenizer=tokenizer,
                                             max_len=max_len_start,
                                             num_workers=num_workers,
)
val_dataloader = make_clustered_dataloader(batch_size,
                                           mega_batch,
                                           val_dataset, 
                                           tokenizer=tokenizer,
                                           max_len=max_len, 
                                           num_workers=1,
)
test_dataloader = make_clustered_dataloader(batch_size,
                                            mega_batch,
                                            test_dataset,
                                            tokenizer=tokenizer,
                                            max_len=max_len, 
                                            num_workers=1,
)
print("length of train dataloader: ", len(train_dataloader))
print("length of val dataloader: ", len(val_dataloader))
print("length of test dataloader: ", len(test_dataloader))

num cpu cores: 16
setting num_workers to 16
length of train dataloader:  21401826
length of val dataloader:  2141
length of test dataloader:  2141


In [3]:
for i,batch in enumerate(train_dataloader):
    print(batch)
    if i == 10:
        break

[[192], [80], [560], [80, 80], [96], [80, 80], [1248], [144]]
tensor([ 192,   80,  560,   80,   96,   80, 1248,  144])
tensor([0, 0, 0, 1, 0, 0, 0, 0])
tensor([[24902455,        0],
        [39637718,        0],
        [16940223,        0],
        [17380753,        1],
        [19223799,        0],
        [10062952,        0],
        [ 2999926,        0],
        [38036935,        0]])
tensor([[39637718,        0],
        [17380753,        1],
        [10062952,        0],
        [19223799,        0],
        [38036935,        0],
        [24902455,        0],
        [16940223,        0],
        [ 2999926,        0]])
tensor([[16940223,        0],
        [ 2999926,        0]])
tensor([[38036935,        0],
        [24902455,        0]])
tensor([[10062952,        0],
        [19223799,        0]])
tensor([[39637718,        0],
        [17380753,        1]])
[[160], [256], [416, 496], [384], [272], [96], [128], [208]]
tensor([160, 256, 496, 384, 272,  96, 128, 208])
tensor([0, 0